# <font color=darkgreen> Building a Deep Neural Network on IMDB dataset-Worksheet-v2.0 </font>

The IMDB dataset consists of 50,000 reviews from the Internet Movie Database split into 50% positive and negative reviews respectively. Also, 25,000 reviews are used for training while the remaining 25,000 for testing.

### <font color=deeppink> Import our essential libraries </font>

In [1]:
# We need os, sys, numpy, tensorflow, matplotlib.pyplot
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

### <font color=deeppink> Loading the IMDB dataset </font>

In [2]:
# IMDB dataset comes pre-packaged with keras
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
# # num_words=10000 will retain only the top 10,000 most frequently occurring words in the training data.
# # this will discard rare words and help us stay focused on the vector data of manageable size.
# # without this limitation, we would end up working with 88,585 unique words in the sample data

In [3]:
# Display the dimensions of the data
# Train data - axes, shape, datatype


In [4]:
# Display the dimensions of the data
# Test images - axes, shape, datatype


In [5]:
# Display a sample review from the dataset
# sample_review = 
# print(sample_review)
# # Each word in the review has been mapped to an index

In [6]:
# Let us print out the lengthiest review in the training data
# lengthy_review = 
# print(lengthy_review)

In [7]:
# Now print out the number of words in the sample review and the lengthy review
# print(f"Number of words in the sample review = {}",
#       f"\nNumber of words in the lengthy review = {}")

In [8]:
# Print the max index in the sample and lengthy review
# print(f"The maximum word index in the sample review = {}",
#      f"\nThe maximum word index in the lengthy review = {}")

# the index will never exceed 9999 since we are looking at the top 10,000 most frequently occurring words in the reviews

In [9]:
# Now repeat loading the dataset into train_data2 and test_data2 and check for the max word index in the lengthiest review.
# Here, we do not restrict to the maximum number of frequently occurring words
(train_data2, train_labels2), (test_data2, test_labels2) = imdb.load_data()
print(train_data2.shape)
print()
# Question: How many total words are there across all the reviews?
# Answer: 

In [10]:
# View the worded review (print these statements one at a time, comment out the rest)
word_index = imdb.get_word_index()
print(word_index)
# The key is the word and the index is the value, hence to get back the word, we need to reverse its order
# reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# print(reverse_word_index)
# sample_worded_review = ' '.join([reverse_word_index.get(i - 3, '-') for i in sample_review])
# print(sample_worded_review)
# lengthy_worded_review = ' '.join([reverse_word_index.get(i - 3 , '?') for i in lengthy_review])
# print(lengthy_worded_review)
# The indices are offset by 3 places because index 0 is reserved for "padding", 
# 1 for "start of the sequence" and 2 for 'unknown"


### <font color=deeppink>Dataset preparation </font>

In this case, we cannot directly feed a list of integers, so we shall one-hot encode (multi-hot encoding) the data
One-hot encoding transforms the list into a vector where only the value of the index where the word occurs would be 1 and rest all 0. 
For example: If we have a review as $[0, 34, 500, 45, 23]$, then one-hot encoding will create a 10,000 dimensional vector (10,000 values) with $1$ only in positions $0, 34, 500, 45, 23$ and the remaining 9,995 positions with values $0$.

In [11]:
# Prepare the dataset for the neural network, we need tensor values - normalized values between 0 to 1
def one_hot_encode_data(review, dimension=10000): 
    encoded_data = np.zeros((len(review), dimension))
    for idx, words in enumerate(review):
        encoded_data[idx, words] = 1. # 1. for a floating point value
    return encoded_data

x_train = one_hot_encode_data(train_data)
x_test = one_hot_encode_data(test_data)

print(x_train.shape)
print(x_test.shape)

In [12]:
x_train[0]

In [13]:
# Convert the labels to numpy arrays and float values like the data
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

print(y_train.shape)
print(y_test.shape)

In [14]:
y_train[0]

### <font color=deeppink> Building the model </font>

In [15]:
# Build the model -- do you recollect?
# We need a Sequential model with 3 Dense layers, with 16 neurons in the first two layers
# and the last with ____ neurons (num_classes) 

model = tf.keras.models.Sequential([
tf.keras.layers.Dense(16, activation='relu', name='Dense1', input_shape=(10000,)),
tf.keras.layers.Dense(16, activation='relu', name='Dense2'),
tf.keras.layers.Dense(1, activation='sigmoid', name='Dense_final')
])
# Later, experiment with 2 Dense layers and 4 Dense layers also vary the hidden neurons (dense layer neurons)


In [16]:
model.summary() # we need to mention the input_shape when defining the model in order to print the summary
# What would be the input_shape?

### <font color=deeppink> Compiling the model </font>

In [17]:
# Compile the model
# Here we shall specify the optimizer, the loss function and the metrics the model needs to focus on
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

### <font color=deeppink> Training the model </font>

In [18]:
# Let us prepare our validation dataset (10,000 reviews from the train dataset)
x_val = x_train[:10000] # reserving the first 10,000 reviews for validation
x_train_small = x_train[10000:] # reserving the remaining 15,000 reviews for training
y_val = y_train[:10000]
y_train_small = y_train[10000:]
# Finally train the model on the train_data and train_labels, epochs=20 and batch_size=512
history = model.fit(x_train_small, y_train_small,
epochs=20,
batch_size=512,
validation_data=(x_val, y_val))

### <font color=deeppink> Plotting the training vs. validation accuracy </font>

In [19]:
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
# plt.savefig('D:/SCIT/MBA-DSDA/Semester-II/Deep Learning-2123/Code Files/Output/imdb_dnn_loss.png')
plt.show()

plt.figure()

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
# plt.savefig('D:/SCIT/MBA-DSDA/Semester-II/Deep Learning-2123/Code Files/Output/imdb_dnn_acc.png')
plt.show()

### <font color=deeppink> Let us predict and see </font>

In [20]:
# Let us predict and see
# y_pred = model.predict_classes(np.expand_dims(x_test[4], axis=0))
# print('Y_pred = {} and Y_true = {}'.format(y_pred, y_test[4]))

# Try building the model by replacing the last Dense layer to have __ input neurons 
# when the activation function=softmax
# What should happen to the loss function? Try with binary_crossentropy, does it work?
# Now recall what you used for the MNIST dataset and check if that loss function works?

# Uncomment and execute the following two lines of code 
y_pred = model.predict(np.expand_dims(x_test[4], axis=0))
print('Y_pred = {} and Y_true = {}'.format(np.argmax(y_pred), y_test[4])) # Try without np.argmax first

# It is time for some more theory! -- go back to the ppt

### <font color=deeppink> Re-train the model for 4 epochs and then predict the results </font>

In [21]:
# Before executing the following code, ensure to re-run the model and its compilation code 
# Try ommitting the above step and you'll see the model's accuracy explode
# This time we train on the complete training set for the ideal number of epochs
history2 = model.fit(x_train, y_train,
epochs=14,
batch_size=512)

In [22]:
# history_dict = history2.history
# loss_value = history_dict['loss']
# val_loss_value = history_dict['val_loss']
# acc = history_dict['accuracy']
# val_acc = history_dict['val_accuracy']
# epochs = range(1, len(loss_value) + 1)
# plt.plot(epochs, loss_value, 'b', label='Training Loss')
# plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# # plt.savefig('D:/SCIT/MBA-DSDA/Semester-II/Deep Learning-2123/Code Files/Output/imdb_dnn_loss.png')
# plt.show()

# plt.figure()

# plt.plot(epochs, acc, 'b', label='Training Accuracy')
# plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# # plt.savefig('D:/SCIT/MBA-DSDA/Semester-II/Deep Learning-2123/Code Files/Output/imdb_dnn_acc.png')
# plt.show()

In [23]:
# Evaluate the model

(loss_model, acc_model) = model.evaluate(x_test, y_test)
# Print the test loss and test accuracy obtained
print(f'Test Loss = {loss_model} and Test Accuracy = {acc_model}')
# Predict and see
results = model.predict(x_test)
results

In [24]:
# Display the confusion matrix
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
# confusion_matrix(test_data, prediction_data)
results2 = model.predict(x_test)
results2 = np.argmax(results2,axis=1)
# confusion_matrix(y_test, np.argmax(results2))
print(results2)

In [25]:
# confusion_matrix?

In [26]:
# Now write a function to display a descriptive confusion matrix
def display_confusion_matrix(actual, pred):
    (tn, fp), (fn, tp) = confusion_matrix(actual, pred)
    print('-----------Confusion Matrix------------')
    print(f'True Positives = {tp} \t True Negatives = {tn}',
         f'\nFale Negatives = {fn} \t False Positives = {fp}')
    print('=======================================')
    print(f'Accuracy = {(tp+tn) / (tp+fn+tn+fp)}')
    
display_confusion_matrix(y_test, results2)

# Homework, complete the function to display recall(TPR), precision and FPR

#### How do we decide the threshold??

# It is time for some more theory! -- go back to the ppt

In [27]:
t = np.arange(0, 1, 0.1)
print(t)
fpr = []
tpr = []
for t1 in t:
    print(f'--Threshold {t1}--')
    pred = np.zeros((y_test.shape))
    for idx, r in enumerate(results):
        if r > t1:
            pred[idx] = 1
    print(pred)
    display_confusion_matrix(y_test, pred)
    (tn, fp), (fn, tp) = confusion_matrix(y_test, pred)
    fpr1 = fp / (fp + tn)
    tpr1 = tp / (tp + fn)
    fpr.append(fpr1)
    tpr.append(tpr1)

In [28]:
print(fpr)
print(tpr)

print('--ROC CURVE-----')
plt.figure()
plt.plot(fpr, tpr)
plt.title('ROC Curve')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [29]:
# Repeat the above exercise for training on 4 epochs with the following changes to the model
# Build a Sequential model with 3 Dense layers, with 31 and 17 neurons each and
# the last with 1 neuron (num_classes)
model3 = tf.keras.models.Sequential([
tf.keras.layers.Dense(31, activation='relu', name='Dense1', input_shape=(10000,)),
tf.keras.layers.Dense(17, activation='relu', name='Dense2'),
tf.keras.layers.Dense(17, activation='relu', name='Dense3'),
# tf.keras.layers.Dropout(0.5, name='Dropout1'),
tf.keras.layers.Dense(1, activation='sigmoid', name='Dense_final')
])
model3.summary()
model3.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])
print('---Model Training----')
model3.fit(x_train, y_train, epochs=4, batch_size=512)
print('---Model Evaluation----')
model3.evaluate(x_test, y_test)
pred3 = model3.predict(x_test)
display_confusion_matrix(y_test, np.argmax(pred3,axis=1))

In [30]:
pred3 = model3.predict(x_test)
display_confusion_matrix(y_test, np.argmax(pred3,axis=-1))

### <font color=deeppink> Dealing with overfitting </font>
#### Add Dropout layers 

# It is time for some more theory! -- go back to the ppt